In [1]:
import getpass
import os

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:")


Tavily API key: ········


In [11]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:")

OpenAI API key: ········


In [3]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False
)


In [83]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.7, model="gpt-4o-mini")  

In [67]:
import requests
from bs4 import BeautifulSoup

def scrape_url(url: str) -> str:
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
        

        return soup.get_text(separator='\n', strip=True)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""


In [77]:
def code_correction_agent(current_code: str, error_message: str, context: str):

    query = f"Correct this code: {current_code}. Error: {error_message}."
    
    search_results = tool.invoke({"query": query})
    

    print("Search Result URLs:")
    urls = [result['url'] for result in search_results]
    for url in urls:
        print(url)
    
 
    scraped_contents = []
    for url in urls[:1]:
        scraped_content = scrape_url(url)
        scraped_contents.append(scraped_content)
    search_content = "\n\n".join(scraped_contents) 
    

    llm_input = [
        {"role": "system", "content": "You are a code correction agent. Your task is to analyze the provided code, identify the reasons for any errors, and suggest a corrected version of the code. Provide a clear explanation of why the original code failed and how the corrected code resolves the issue."},
        {"role": "user", "content": f"Current Code:\n{current_code}\n\nError Message:\n{error_message}\n\nContext from Scraped URLs:\n{context}\n\nRelevant Search Result:\n{search_content}\n\nPlease provide an explanation of the failure and the corrected code:"}
    ]
    
    corrected_code = llm.invoke(llm_input)
    
    return corrected_code

In [85]:
current_code = """
def add_numbers(a, b):
    return a + b
"""

error_message = "TypeError: unsupported operand type(s) for +: 'int' and 'str'"
context = "This function is supposed to add two numbers."

corrected_code = code_correction_agent(current_code, error_message, context)


print("Corrected Code and Explanation:")
print(corrected_code)


Search Result URLs:
https://sebhastian.com/typeerror-unsupported-operand-types-slash-for-str-and-int/
https://stackoverflow.com/questions/20441035/unsupported-operand-types-for-int-and-str
https://researchdatapod.com/python-unsupported-operand-types-for-str-and-int/
https://www.geeksforgeeks.org/python-typeerror-unsupported-operand-type/
https://www.geeksforgeeks.org/how-to-fix-typeerror-unsupported-operand-types-for-nonetype-and-str-in-python/
Corrected Code and Explanation:
content='### Explanation of the Failure\n\nThe `TypeError: unsupported operand type(s) for +: \'int\' and \'str\'` error occurs when attempting to add an integer (`int`) and a string (`str`) together in Python. This typically happens when user input is involved, as the `input()` function in Python always returns data as a string, regardless of what the user enters. \n\nIn your original function `add_numbers(a, b)`, if either `a` or `b` is a string, and the other is an integer, Python cannot perform the addition op